In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# X_train = (Ear Shape = (1 if Pointy, 0 if floppy),    Face Shape = (1 if round, 0 if not round),      Whiskers = (1 if Present, 0 if Absent))

features = ["Ear Shape", "Face Shape", "Whiskers"]

X_train = np.array(
    [
        [1, 1, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 1],
        [1, 1, 1],
        [1, 1, 0],
        [0, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
    ]
)

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [3]:
X_train[0]

array([1, 1, 1])

In [4]:
# Entropy

def entropy(p) -> float:
    if (p == 0 or p == 1):
        return 0
    
    return - (p * np.log2(p) + (1 - p) * np.log2(1 - p))

print(entropy(0.5))

1.0


In [5]:
def split_indices(X, feature_index: int):
    """For a given feature (index of the training set (one hot encoded)) returns the indices of features that satisfy the criteria and vice versa

    Args:
        X (Numpy Array) : Dataset
        feature_index (int): Index of feature

    Returns:
        tuple: left tree values and right tree values
    """
    
    left_tree = []
    right_tree = []

    for i, x in enumerate(X):
        if (x[feature_index] == 1):
            left_tree.append(i)
        else:
            right_tree.append(i)

    return left_tree, right_tree


In [6]:
split_indices(X_train, 0)

([0, 3, 4, 5, 7], [1, 2, 6, 8, 9])

In [ ]:
def weighted_entropy(X, y, left_tree, right_tree) -> float:
    """
    This function takes the splitted dataset, the indices we chose to split and returns the weighted entropy.
    """
    prob_left = 0
    w_left = len(left_tree) / len(X)
    if (len(left_tree) > 0):
        prob_left = sum(y[left_tree]) / len(left_tree)

    prob_right  = 0
    w_right = len(right_tree) / len(X)
    if (len(right_tree) > 0):
        prob_right = sum(y[right_tree]) / len(right_tree)

    weighted_entropy = w_left * entropy(prob_left) + w_right * entropy(prob_right)
    return weighted_entropy

: 

In [107]:
left_tree, right_tree = split_indices(X_train, 0)
weighted_entropy(X_train, y_train, left_tree, right_tree)

0.7219280948873623

In [108]:
def information_gain(X, y, left_tree, right_tree) -> float:
    p_node = sum(y) / len(y)
    p_node_entropy = entropy(p_node)
    c_entropy = weighted_entropy(X, y, left_tree, right_tree)

    return p_node_entropy - c_entropy

In [109]:
information_gain(X_train, y_train, left_tree, right_tree)

0.2780719051126377

In [110]:
def split(X, y) -> tuple[int, float, list[int], list[int]]:
    if len(X) == 0 or len(y) == 0:
        return -1, 0, [], []
    
    max_info_gain: float = 0
    split_index = 0
    max_left_tree = max_right_tree = []

    for i in range(X.shape[1]): # columns
        left_tree, right_tree = split_indices(X, i)
        info_gain = information_gain(X, y, left_tree, right_tree)
        if (info_gain > max_info_gain):
            split_index = i
            max_info_gain = info_gain
            max_left_tree = left_tree
            max_right_tree = right_tree

    return split_index, max_info_gain, max_left_tree, max_right_tree

In [118]:
def build_tree(X, y, i, max_depth: int):
    if (max_depth == 0):
        return
    
    split_index, info_gain, left_tree, right_tree = split(X, y)
    if (info_gain < 1e-3):
        return
    
    left_indices = i[left_tree]
    right_indices = i[right_tree]
    print("splitting on : ", features[split_index])
    print(left_indices, right_indices)

    build_tree(X[left_tree], y[left_tree], left_indices, max_depth - 1)
    build_tree(X[right_tree], y[right_tree], right_indices, max_depth - 1)

print(build_tree(X_train, y_train, np.arange(X_train.shape[0]), 2))

splitting on :  Ear Shape
[0 3 4 5 7] [1 2 6 8 9]
splitting on :  Face Shape
[0 4 5 7] [3]
splitting on :  Whiskers
[1] [2 6 8 9]
None
